In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [35]:
# requests used to get the URL
URL = "https://forecast.weather.gov/MapClick.php?x=276&y=148&site=lox&zmx=&zmy=&map_x=276&map_y=148"
page = requests.get(URL)
page

<Response [200]>

In [36]:
soup = BeautifulSoup(page.content, "html.parser")

In [37]:
# extract the day of the week from the given website

day = []       # create a new list for insert the days
# find_all used to find all the information in the website, but in the find_all we can not use the .text
# In the find only we can use the .text
# .text used to remove the extra things 
weather_day = soup.find_all('div', class_ ='col-sm-2 forecast-label')             # to use the.text we are using the for loop
for days in weather_day[1 : 11]:
  day.append(days.text)
print(day)

len(day)

['Sunday', 'Sunday Night', 'Monday', 'Monday Night', 'Tuesday', 'Tuesday Night', 'Wednesday', 'Wednesday Night', 'Thursday', 'Thursday Night']


10

In [38]:
# extract the description
desc = []
weather_desc = soup.find_all('div', class_ ='col-sm-10 forecast-text')
for description in weather_desc[1 : 11]:
  desc.append(description.text)
print(desc)

len(desc)

['Mostly sunny, with a high near 72. East wind 5 to 10 mph becoming west 10 to 15 mph in the afternoon. Winds could gust as high as 20 mph. ', 'Mostly clear, with a low around 50. Northwest wind 5 to 10 mph becoming east northeast after midnight. Winds could gust as high as 15 mph. ', 'Sunny, with a high near 76. Northeast wind 5 to 15 mph becoming southwest in the afternoon. ', 'Mostly clear, with a low around 54. West wind around 5 mph becoming northeast after midnight. ', 'Mostly sunny, with a high near 76.', 'Partly cloudy, with a low around 52.', 'Sunny, with a high near 79. Breezy. ', 'Mostly clear, with a low around 54.', 'Mostly sunny, with a high near 81.', 'Mostly clear, with a low around 55.']


10

In [39]:
# date of the day
dates = pd.date_range(start= pd.to_datetime("today"), periods = 5)

# account for day and night pattern (repeating the same date twice)
day_night = []
for a, b in zip(dates, dates):        # zip used to repeat the things
    day_night.append(a.date())
    day_night.append(b.date())
print(day_night)

len(day_night)

[datetime.date(2022, 3, 13), datetime.date(2022, 3, 13), datetime.date(2022, 3, 14), datetime.date(2022, 3, 14), datetime.date(2022, 3, 15), datetime.date(2022, 3, 15), datetime.date(2022, 3, 16), datetime.date(2022, 3, 16), datetime.date(2022, 3, 17), datetime.date(2022, 3, 17)]


10

In [40]:
# extract the temperature from the description 
temp = []
for i in desc:
    temp.append(re.findall('[0-9]+',(i.split(',')[1].split('.')[0])))     #Finding the number in the sentence
print(temp)

# packing a list of list
temp = [degree for list in temp for degree in list]
print(temp)
#converting list of strings into an integer
temp = [int(i) for i in temp]
print(temp)


[['72'], ['50'], ['76'], ['54'], ['76'], ['52'], ['79'], ['54'], ['81'], ['55']]
['72', '50', '76', '54', '76', '52', '79', '54', '81', '55']
[72, 50, 76, 54, 76, 52, 79, 54, 81, 55]


In [41]:
# To convert Fahrenheit to Celsius
# Formula	
# (32°F − 32) × 5/9 = 0°C
converted_temp =[]
for i in temp:
    converted_temp.append(round((i-32)*(5/9),2))        
print(converted_temp)

[22.22, 10.0, 24.44, 12.22, 24.44, 11.11, 26.11, 12.22, 27.22, 12.78]


In [42]:
# join scraped data into a dataframe

weather_data = pd.DataFrame({ 'Date' : day_night,
               'Day': day,
               'Temperature': converted_temp,
               'Description': desc})

weather_data

,Date,Day,Temperature,Description
0,2022-03-13,Sunday,22.22,"Mostly sunny, with a high near 72. East wind 5..."
1,2022-03-13,Sunday Night,10.00,"Mostly clear, with a low around 50. Northwest ..."
2,2022-03-14,Monday,24.44,"Sunny, with a high near 76. Northeast wind 5 t..."
3,2022-03-14,Monday Night,12.22,"Mostly clear, with a low around 54. West wind ..."
4,2022-03-15,Tuesday,24.44,"Mostly sunny, with a high near 76."
5,2022-03-15,Tuesday Night,11.11,"Partly cloudy, with a low around 52."
6,2022-03-16,Wednesday,26.11,"Sunny, with a high near 79. Breezy."
7,2022-03-16,Wednesday Night,12.22,"Mostly clear, with a low around 54."
8,2022-03-17,Thursday,27.22,"Mostly sunny, with a high near 81."
9,2022-03-17,Thursday Night,12.78,"Mostly clear, with a low around 55."
